# Build the get_lipidomics_data() function for dash app and general querying

Adapting from `src/dash/data.py` function `get_lipidomics_data()`

In [1]:
from sqlalchemy import create_engine, MetaData, Table, select, join
import pandas as pd

# SQLite path (updated to include new version of lipidomics data)
db_path = 'sqlite:///../../../data/SQLite Database/20200524/Covid-19 Study DB.sqlite'


def get_lipidomics_data(with_metadata=False):
    # Create an engine that connects to the Covid-19 Study DB.sqlite file: engine
    engine = create_engine(db_path)

    # Establish connection
    connection = engine.connect()

    # pull table into df
    lipidomics_measurements_df = pd.read_sql_query("SELECT * from lipidomics_measurements", connection)

    # pull table into df
    lipidomics_runs_df = pd.read_sql_query("SELECT * from lipidomics_runs", connection)

    # pull table into df
    rawfiles_df = pd.read_sql_query("SELECT * from rawfiles WHERE ome_id=2 AND sample_ID<>-1 and keep=1", connection)

    # pull table into df
    deidentified_patient_metadata_df = pd.read_sql_query("SELECT * from deidentified_patient_metadata", connection)

    # make sure the merge by columns are all the same type -> pandas seems sensitive to this
    lipidomics_measurements_df = lipidomics_measurements_df.astype({'replicate_id': 'int32'})
    lipidomics_runs_df = lipidomics_runs_df.astype({'replicate_id': 'int32', 'rawfile_id': 'int32'})
    rawfiles_df = rawfiles_df.astype({'rawfile_id': 'int32', 'sample_id': 'int32'})
    deidentified_patient_metadata_df = deidentified_patient_metadata_df.astype({'sample_id': 'int32'})

    joined_df = lipidomics_measurements_df\
                .join(lipidomics_runs_df.set_index('replicate_id'), on='replicate_id')\
                .join(rawfiles_df.set_index('rawfile_id'), on='rawfile_id')\
                .join(deidentified_patient_metadata_df.set_index('sample_id'), on='sample_id')

    # drop samples that are missing COVID or ICU status
    joined_df.dropna(subset=['ICU_1','COVID'], inplace=True)

    # pivot to wide format
    wide_df = joined_df.pivot_table(index='sample_id', columns='biomolecule_id', values='normalized_abundance')
    wide_df.columns = [str(col) for col in wide_df.columns]

    # get biomolecule names
    biomolecules_df = pd.read_sql_query("SELECT * from biomolecules WHERE keep=1 AND omics_id=2", connection)
    
    # close DB connection
    connection.close()

    # build biomolecule name dict 
    biomolecule_name_dict = {}
    for index, row in biomolecules_df.iterrows():
        biomolecule_id = str(row['biomolecule_id'])
        standardized_name = row['standardized_name']
        biomolecule_name_dict[biomolecule_id] = standardized_name

    # replace wide_df column names
    new_col_names = []
    for col in wide_df.columns:
        new_col_names.append(biomolecule_name_dict[str(col)])
    wide_df.columns = new_col_names

    # optional return matrix with clinical metadata
    if with_metadata:

        combined_df = wide_df.join(deidentified_patient_metadata_df.set_index('sample_id'), on='sample_id')#.dropna()
        return combined_df
    
    return wide_df

In [2]:
lididomics_df = get_lipidomics_data(with_metadata=False)
lididomics_df.head()

,Unknown Lipid RT1.093 +_Duplicate,Unknown Lipid RT1.094 +_Duplicate,Unknown Lipid RT1.107 +_Duplicate,Unknown Lipid RT1.109 +_Duplicate,Unknown Lipid RT1.119 -_Duplicate,Unknown Lipid RT1.119 +_Duplicate,Unknown Lipid RT1.120 -_Duplicate,Unknown Lipid RT1.121 -_Duplicate,Unknown Lipid RT1.122 -_Duplicate,Unknown Lipid RT1.122 -_Duplicate,...,Unknown Lipid RT35.174 +,TG 18:1_18:1_24:0 +,Unknown Lipid RT35.179 +,Unknown Lipid RT35.373 +,Unknown Lipid RT35.391 +,Unknown Lipid RT35.392 +,TG 60:1 +,Unknown Lipid RT35.592 +,Unknown Lipid RT42.382 +,Unknown Lipid RT42.388 -
sample_id,,,,,,,,,,,,,,,,,,,,,
1.0,15.976462,19.033580,20.875503,16.584828,16.566370,15.861481,19.601650,15.468000,18.107826,16.353128,...,15.282698,17.996415,14.874285,15.054255,14.150701,14.073726,16.298505,15.216885,16.091088,15.733284
2.0,13.687458,19.211487,20.978229,18.217534,18.007366,15.790173,17.426756,16.879111,17.960593,17.251460,...,14.568996,16.871878,14.586308,14.079343,14.638520,12.953482,14.994721,14.302267,16.351197,15.998386
3.0,17.979818,18.662757,20.678124,17.964309,15.720253,16.016798,17.037753,15.689912,17.204381,16.119386,...,17.223982,19.712136,16.513870,15.507988,15.152690,13.634892,17.583291,16.667818,15.977472,16.576950
4.0,17.467381,19.262132,20.969724,17.733312,15.043289,15.084114,19.073448,17.482569,17.209061,16.133293,...,17.328677,20.166055,17.689385,16.094259,15.617449,14.919465,18.043384,17.263973,15.385761,16.650355
5.0,15.554856,18.942669,20.718091,18.071819,18.516299,18.096767,16.834873,15.669407,17.200442,15.690189,...,17.156789,19.899135,17.430503,16.170194,15.293360,14.859790,17.946729,17.179654,15.176656,15.532393
